In [1]:
import numpy as np
import pandas as pd
from numpy import linalg as LA

In [2]:
# Split a dataset based on an attribute and an attribute value
def test_split(index, value, dataset):
    left, right = list(), list()
    for row in dataset:
        if row[index] < value:
            left.append(row)
        else:
            right.append(row)
    return left, right

In [3]:
# Create a terminal node value
def to_terminal(group):
    outcomes = [row[-1] for row in group]
    return max(set(outcomes), key=outcomes.count)

In [4]:
# Create child splits for a node or make terminal
def split(node, max_depth, min_size, depth):
    left, right = node[ ' group ' ]
    #print(left[0][len(left[0])-1])
    #print(right[0][len(right[0])-1])
    del(node[ ' group ' ])
    # check for a no split
    if not left or not right:
        node[ ' left ' ] = node[ ' right ' ] = to_terminal(left + right)
        return
    # check for max depth
    if depth >= max_depth:
        node[ ' left ' ], node[ ' right ' ] = to_terminal(left), to_terminal(right)
        return
    # process left child
    if len(left) <= min_size:
        node[ ' left ' ] = to_terminal(left)
    #elif purity(left):
    #    x= 10
    else:
        node[ ' left ' ] = get_split(left)
        if(node[ ' left ' ]!= 0):
            split(node[ ' left ' ], max_depth, min_size, depth+1)
        #else:
        #    node[ ' left ' ]= { ' index ' : 0, ' value ' :left[0][len(left[0])-1], ' group ' :0}
    # process right child
    if len(right) <= min_size:
        node[ ' right ' ] = to_terminal(right)
    else:
        node[ ' right ' ] = get_split(right)
        if(node[ ' right ' ]!= 0):
            split(node[ ' right ' ], max_depth, min_size, depth+1)
        #else:
        #    node[ ' right ' ]= { ' index ' : 0, ' value ' :right[0][len(right[0])-1], ' group ' :0}

In [5]:
# Select the best split point for a dataset
def get_split(dataset):
    data= pd.DataFrame(dataset)
    minBC = 9999
    index= 0
    for col_num in range(len(data.columns)-1):
        cols = [col_num, len(data.columns)-1]
        var = data[cols]
        New_data= pd.DataFrame(var)
        categories = list(New_data[cols[0]].unique())
        classes = list(var[cols[1]].unique())
        if(len(classes)== 1):
            return 0;
        count_matrix ={}
        for cl in classes:
            count_dict = {}
            for ct in categories:
                cell = var[(var[cols[0]] == ct) & (var[cols[1]] == cl)]
                count_dict[ct] = cell.shape[0]
            count_matrix[cl] = count_dict
        class_distribution_matrix = pd.DataFrame(count_matrix)
        if(class_distribution_matrix.shape[0]!= 1):
            class_distribution_array= class_distribution_matrix.iloc[:, :class_distribution_matrix.shape[1]].values
            N= data.shape[0]
            NV= len(categories)
            CA= N/NV
            EP= 0
            for variable in range(NV):
                sum= 0
                for num in range(class_distribution_matrix.shape[1]):
                    sum= sum+ class_distribution_array[variable][num]
                result= CA- sum
                if(result< 0):
                    result= result* -1
                EP= EP+result
            #At this particular point, we got EP (Equal Split Parameter)
            covariance_matrix= class_distribution_matrix.cov()
            eig_vals, eig_vecs = LA.eig(covariance_matrix)
            
            sum= 0
            for value in range(len(eig_vals)):
                sum= sum+ eig_vals[value]
            average= sum/len(eig_vals)
            
            BC= EP/average
            print(BC)
            if(BC< minBC):
                minBC= BC
                index= col_num
                matrix= class_distribution_matrix
    print(minBC)
    print(index)
    print(matrix)
    if matrix.shape[0]== 2:
        value= matrix.index.values[0]
        #print('value= ', value)
    #else:
        #find impurity
        
    value= dataset[0][index]
    group= test_split(index, value, dataset)
    return { ' index ' :index, ' value ' :value, ' group ' :group}

In [6]:
# Build a decision tree
def build_tree(train, max_depth, min_size):
    root = get_split(dataset)
    split(root, max_depth, min_size, 1)
    return root

In [7]:
# Print a decision tree
def print_tree(node, depth=0):
    #print(node)
    #if node[ ' group ' ]== 0:
    #    print('['+node['value']+']')
    #    return 0
    if isinstance(node, dict):
        print( ' %s[Var0%d == %s] ' % ((depth* ' ' , (node[ ' index ' ]+1), node[ ' value ' ])))
        print_tree(node[ ' left ' ], depth+1)
            
        print_tree(node[ ' right ' ], depth+1)
    else:
        print( ' %s[%s] ' % ((depth* ' ' , node)))

In [8]:
data = pd.read_csv('Sample Dataset.csv')
dataset= data.iloc[:, :4].values
tree = build_tree(dataset,2, 1)
print_tree(tree)

2.2857142857142865
1.9999999999999998
0.23529411764705882
0.23529411764705882
2
             1  2
Round        5  1
Rectangular  0  4
1.4999999999999998
4.8
1.4999999999999998
0
       1  2
White  3  0
Black  2  0
Pink   0  1
 [Var03 == Round] 
  [0] 
  [Var01 == White] 
   [1] 
   [1] 
